<a href="https://colab.research.google.com/github/aminghazanfari83/Python/blob/master/Modulation_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install tensorflow-gpu==2.0.0

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
tf.__version__

'2.0.0'

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
!ls '/gdrive/My Drive/Colab Notebooks'

'Copy of FGM_FGSM_Image_classification_cnn (1).ipynb'
'Copy of FGM_FGSM_Image_classification_cnn.ipynb'
'Copy of I-FGSM_MI-FGSM.ipynb'
'Copy of Localization_model.ipynb'
 dataset_maxmin.mat
 dataset_maxprod.mat
 Demo_Comm_Sys.ipynb
 FGM_FGSM_Image_classification_cnn.ipynb
 FGV.ipynb
 h_Estimated_CTW_Train.mat
 I-FGSM_MI-FGSM.ipynb
 Image_classification_cnn.ipynb
 Localization_model.ipynb
 Modulation_classification.ipynb
 NN_MMMSE_maxprod_manoj.ipynb
 pow_MMMSE_maxmin_cell_1.mat
' Ref_Localization_model.ipynb'
 RML2016.10a_dict.pkl
 r_Position_CTW_Train.mat
 set_Features.mat
 set_Labels.mat
 SNR_CTW_Train.mat
 testset_maxmin.mat
 testset_maxprod.mat
 tf_NN_MMMSE_maxprod_manoj.ipynb
 Untitled0.ipynb
 Untitled1.ipynb
 Untitled2.ipynb
 Untitled3.ipynb
 Untitled4.ipynb
 Untitled5.ipynb


In [ ]:
    with open('/gdrive/My Drive/Colab Notebooks/RML2016.10a_dict.pkl', 'rb') as open_file:
        u = pickle._Unpickler( open_file )
        u.encoding = 'latin1'
        Xd = u.load() # a dict with 220 keys of the form ('modulation_name', snr_value) where modulation_name is in {'8PSK','AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM'} (11 mod) and snr_value is in {-20, -18, ..., 18} (20 snr_value)

In [ ]:
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])

In [ ]:
  X = []  
  lbl = []
  for mod in mods:
      for snr in snrs:
          X.append(Xd[(mod,snr)])
          for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr)) # Doubt: Only one modulation and one snr is labelled
  X = np.vstack(X)

In [ ]:
mods.index('8PSK')
print(X.shape)
print(X[0,:].shape)

(220000, 2, 128)
(2, 128)


In [ ]:
np.random.seed(2016)
n_examples = X.shape[0]
n_train = int(n_examples * 0.5)
train_idx = np.random.choice(range(0,n_examples), size=n_train, replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]
print(train_idx)
def to_onehot(yin):
    yy = list(yin)       # Doubt: Is list required
    yy1 = np.zeros([len(list(yy)), max(yy)+1])
    yy1[np.arange(len(list(yy))),yy] = 1
    return yy1
Y_train = to_onehot(map(lambda x: mods.index(lbl[x][0]), train_idx)) # Doubt 
Y_test = to_onehot(map(lambda x: mods.index(lbl[x][0]), test_idx))

[ 83808  53552 145413 ... 101817  52243 162017]


In [ ]:
Y_test[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
X_train.shape[1:]
train_SNRs = list(map(lambda x: lbl[x][1], train_idx))
print(train_SNRs)
train_Y_0 = Y_train[np.where(np.array(train_SNRs)== 12)] 
print(train_Y_0)


[-14, 6, -10, -14, -12, 4, -2, 18, -10, -2, 16, -12, 16, 16, 12, -6, 18, 14, 10, 8, 12, -4, 8, 2, 18, 0, -2, -10, -16, -10, 8, -6, -4, 8, -20, 2, 12, -16, -2, -2, 2, -12, 8, -14, -8, 16, -20, 0, -4, 18, -10, -18, 6, 2, 12, 10, 10, 4, 16, -8, -8, -18, 2, 18, -16, 2, -16, 0, -4, 4, -16, -12, 6, 4, -20, -20, -20, 6, -4, -16, -20, 16, 10, -2, -8, 6, -10, 10, -8, 0, 0, -10, -4, -2, 16, -20, -20, -12, 14, -12, 6, 2, -16, -4, 0, -20, 10, -6, -18, 18, -20, -2, -20, 8, 16, 8, 4, 8, 14, 10, -4, 6, 14, 14, 4, -6, 4, 14, -8, 16, -14, 18, 0, -6, 8, 14, -6, -4, 8, -2, 4, 16, -8, 10, -6, -2, 0, -6, -4, -16, -2, 6, 4, 4, -14, 12, 6, -14, -14, -20, -2, 16, -14, 0, -6, 14, -2, -20, -14, -6, -2, -20, -20, -18, -14, -16, -16, -6, 2, -2, 8, 4, -4, -2, -14, 12, 18, 10, 14, 16, 0, -18, 18, 18, 14, 10, -20, -10, -8, 8, -14, 18, -12, -18, -6, 18, -16, 10, 12, -16, 10, -14, -14, -8, -6, -8, 8, 0, -2, 8, -14, 2, 12, 14, -14, -16, -14, 8, 4, 4, -4, -6, -20, 8, 10, -2, -8, 10, 14, 2, 14, -4, -18, -18, 0, 16, 18, 0

In [ ]:
in_shp = list(X_train.shape[1:]) # Doubt: Why list 
classes = mods
print (in_shp)

[2, 128]


In [ ]:
print('X_train.shape:', X_train.shape)
print('Y_train.shape:', Y_train.shape)
print(X.shape)
print(len(lbl))

print('Y_train[4:9]: \n', Y_train[4:9])
print('train_idx.shape:', train_idx.shape)
print('train_idx[4:9]:', train_idx[4:9])
print(lbl[train_idx[4]], lbl[train_idx[5]], lbl[train_idx[6]], lbl[train_idx[7]], lbl[train_idx[8]], lbl[train_idx[9]])
print('classes:', classes)

X_train.shape: (110000, 2, 128)
Y_train.shape: (110000, 11)
(220000, 2, 128)
220000
Y_train[4:9]: 
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]
train_idx.shape: (110000,)
train_idx[4:9]: [184829 192624 169402 179790 145554]
('QPSK', -12) ('QPSK', 4) ('QAM64', -2) ('QAM64', 18) ('QAM16', -10) ('BPSK', -2)
classes: ['8PSK', 'AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM']


In [ ]:
# The CNN Network
list(X_train.shape[1:])

in_shp = list(X_train.shape[1:])
dr = 0.5 # dropout rate (%)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Reshape([1]+in_shp, input_shape=in_shp))
model.add(tf.keras.layers.ZeroPadding2D((0,2),data_format='channels_first'))
model.add(tf.keras.layers.Convolution2D(256,(1,3), padding='valid', activation="relu", name="conv1",data_format='channels_first', kernel_initializer='glorot_uniform'))
model.add(tf.keras.layers.Dropout(dr))
model.add(tf.keras.layers.ZeroPadding2D((0,2),data_format='channels_first'))
model.add(tf.keras.layers.Convolution2D(80,(2,3), padding="valid", activation="relu", name="conv2", data_format='channels_first', kernel_initializer='glorot_uniform'))
model.add(tf.keras.layers.Dropout(dr))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal', name="dense1"))
model.add(tf.keras.layers.Dropout(dr))
model.add(tf.keras.layers.Dense(len(classes), kernel_initializer='he_normal', name="dense2" ))
model.add(tf.keras.layers.Activation('softmax')) # Doubt: activation could have been inside in the previous line
model.add(tf.keras.layers.Reshape([len(classes)])) # Doubt: Why this reshape
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 1, 2, 128)         0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 1, 2, 132)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 256, 2, 130)       1024      
_________________________________________________________________
dropout (Dropout)            (None, 256, 2, 130)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 256, 2, 134)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 80, 1, 132)        122960    
_________________________________________________________________
dropout_1 (Dropout)          (None, 80, 1, 132)        0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls ./drive

In [ ]:
in_shp

[2, 128]

In [ ]:
# Set up some params 
nb_epoch = 100    # number of epochs to train on
batch_size = 1024  # training batch size
filepath = "./drive/My Drive/new_model.h5"
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
]
# perform training ...
history = model.fit(X_train,
    Y_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=2,
    validation_data=(X_test, Y_test),
    callbacks = callbacks)

In [ ]:
#evaluate(x=None, y=None, batch_size=None, verbose=1, sample_weight=None, steps=None, callbacks=None, max_queue_size=10, workers=1, use_multiprocessing=False)

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=1, batch_size=batch_size)
print(score)

In [ ]:
# Show loss curves 
plt.figure()
plt.title('Training performance')
plt.plot(history.epoch, history.history['loss'], label='train loss+error')
plt.plot(history.epoch, history.history['val_loss'], label='val_error')
plt.legend()